In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
athletes = pd.read_csv('athlete_events.csv')
athletes.shape

(271116, 15)

In [29]:
Population = pd.read_csv('WorldPopulation.csv')
Population.shape

(217, 62)

In [30]:
Population.drop(['Indicator Name', 'Indicator Code', 'Unnamed: 61'], axis = 1, inplace=True)

In [31]:
Population.sample(3)

,Country,Country Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
214,South Africa,ZAF,17396367.0,17850045.0,18322335.0,18809939.0,19308166.0,19813947.0,20325230.0,20843785.0,...,4.888384e+07,4.955757e+07,5.025581e+07,5.097943e+07,5.172935e+07,5.250652e+07,5.331196e+07,5.414673e+07,5.501198e+07,55908865
171,Solomon Islands,SLB,117866.0,121396.0,125064.0,128866.0,132782.0,136847.0,141026.0,145351.0,...,4.929400e+05,5.044770e+05,5.160790e+05,5.277900e+05,5.396140e+05,5.515310e+05,5.635130e+05,5.755040e+05,5.874820e+05,599419
119,St. Martin (French part),MAF,4279.0,4453.0,4566.0,4656.0,4748.0,4841.0,4936.0,5033.0,...,2.890500e+04,2.937600e+04,2.982000e+04,3.023500e+04,3.061500e+04,3.095900e+04,3.126400e+04,3.153000e+04,3.175400e+04,31949


In [32]:
athletes.sample(3)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
137481,69113,Jean Georges Levavasseur,M,24.0,NaN,NaN,France,FRA,1948 Summer,1948,Summer,London,Fencing,"Fencing Men's Sabre, Team",NaN
27103,14098,Thierry Bourdin,M,21.0,160.0,57.0,France,FRA,1984 Summer,1984,Summer,Los Angeles,Wrestling,"Wrestling Men's Flyweight, Freestyle",NaN
179271,90077,Julien Ouedraogo,M,26.0,175.0,73.0,Burkina Faso,BUR,2008 Summer,2008,Summer,Beijing,Fencing,"Fencing Men's Sabre, Individual",NaN


In [33]:
city = athletes['City'] == 'Beijing'
athletes[city].head(3)
athletes[city].shape

(13602, 15)

In [34]:
Population = pd.melt(Population, id_vars = ['Country', 'Country Code'], var_name = 'Year', value_name = 'Population' )

In [35]:
Population['Year'] = pd.to_numeric(Population['Year'])

In [37]:
Population.head(3)

,Country,Country Code,Year,Population
0,Aruba,ABW,1960,54211.0
1,Afghanistan,AFG,1960,8996351.0
2,Angola,AGO,1960,5643182.0


In [8]:
#a = athletes.set_index('NOC').join(population_2008.set_index('Country Code'), how='inner')

In [38]:
Population = pd.merge(Population, athletes, left_on = ['Country Code', 'Year'], right_on = ['NOC', 'Year'])

In [39]:
Population.shape

(151931, 18)

In [40]:
Population.head(4)

,Country,Country Code,Year,Population,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Season,City,Sport,Event,Medal
0,Afghanistan,AFG,1960,8996351.0,29626,Sultan Mohammad Dost,M,28.0,168.0,73.0,Afghanistan,AFG,1960 Summer,Summer,Roma,Wrestling,"Wrestling Men's Welterweight, Freestyle",NaN
1,Afghanistan,AFG,1960,8996351.0,39922,Abdul Ghafar Ghafoori,M,22.0,172.0,70.0,Afghanistan,AFG,1960 Summer,Summer,Roma,Athletics,Athletics Men's 4 x 100 metres Relay,NaN
2,Afghanistan,AFG,1960,8996351.0,43901,Ghulam Mohiddin Gunga,M,26.0,168.0,81.0,Afghanistan,AFG,1960 Summer,Summer,Roma,Wrestling,"Wrestling Men's Light-Heavyweight, Freestyle",NaN
3,Afghanistan,AFG,1960,8996351.0,58364,Mohammad Ibrahim Kederi,M,20.0,166.0,62.0,Afghanistan,AFG,1960 Summer,Summer,Roma,Wrestling,"Wrestling Men's Featherweight, Freestyle",NaN


In [41]:
Population.to_csv('athletePopulation.csv', sep=',')